In [10]:
import os,glob
import numpy as np
import SimpleITK as sitk
from math import fabs, copysign
from collections import OrderedDict
import shutil
from utils import *

In [2]:
data_file = '/media/ubuntu/Seagate Expansion Drive/IACTA/xianjin/Medzoo_code_use_data/nii'

In [3]:
data_list = sorted(os.listdir(os.path.join(data_file,'cta_img')))
print (len(data_list))
print (data_list[0])

566
14409.nii.gz


In [4]:
def generate_datainfo_csv(name,IA_voxels,IA_minz,slices):
 
    #name_attribute = ['id','slices','IA_voxels','IA_minz','IA_size']

    img = dict()
    img['id'] = name
    img['slices'] = slices
    img['IA_voxels'] = IA_voxels
    img['IA_minz'] = IA_minz
    
    x = img['IA_voxels']
    y = copysign(fabs(x) ** (1 / 3), x)
    
    img['IA_size'] = int(y)
    
    return img

In [5]:
def generate_datainfo_pkl(name,cta,seg,data_file):

    property = OrderedDict()
    property['id'] = name
    property['ori_cta'] = os.path.join(data_file, 'cta_img', name + '.nii.gz')
    property['ori_seg'] = os.path.join(data_file, 'ane_seg', name + '.nii.gz')
    
    data_itk = cta
    property["original_size_of_raw_data"] = np.array(data_itk.GetSize())[[2, 1, 0]]
    property["original_spacing"] = np.array(data_itk.GetSpacing())[[2, 1, 0]]
    property["itk_origin"] = data_itk.GetOrigin()
    property["itk_spacing"] = data_itk.GetSpacing()
    property["itk_direction"] = data_itk.GetDirection()
    
    seg_np = sitk.GetArrayFromImage(seg).astype(np.int32)
    property['slices'] = seg_np.shape[0]
    property['IA_voxels'] = seg_np.sum()
    IA_minz = np.min(np.where(seg_np==1)[0])
    property['IA_minz'] = int(IA_minz)

    return property,property['IA_voxels'],property['IA_minz'],property['slices']


In [8]:
csv_info_list = []
all_properties = OrderedDict()

for data in data_list:
    name = data[:-7]
    print ('name:',name)
    cta_itk = sitk.ReadImage(os.path.join(data_file,'cta_img',data))
    seg_itk = sitk.ReadImage(os.path.join(data_file,'ane_seg',data))
    
    #pkl
    properti,IA_voxels,IA_minz,slices = generate_datainfo_pkl(name,cta_itk,seg_itk,data_file)
    print ('IA_minz:',IA_minz)
    print ('slices:',slices)
    print ()
    all_properties[name] = properti
    
    
    #csv
    csv_info_list.append(generate_datainfo_csv(name,IA_voxels,IA_minz,slices))
    


name: 14409
IA_minz: 390
slices: 557

name: 151667
IA_minz: 94
slices: 265

name: 198420
IA_minz: 353
slices: 579

name: 2174
IA_minz: 579
slices: 771

name: 240040
IA_minz: 159
slices: 301

name: 271870
IA_minz: 124
slices: 222

name: 277702
IA_minz: 207
slices: 298

name: 28272
IA_minz: 257
slices: 557

name: 283534
IA_minz: 278
slices: 387

name: 293250
IA_minz: 512
slices: 699

name: 294627
IA_minz: 539
slices: 699

name: 298438
IA_minz: 507
slices: 699

name: 298447
IA_minz: 304
slices: 601

name: 299916
IA_minz: 54
slices: 176

name: 302691
IA_minz: 177
slices: 353

name: 303700
IA_minz: 528
slices: 712

name: 308477
IA_minz: 496
slices: 699

name: 312521
IA_minz: 99
slices: 265

name: 316888
IA_minz: 93
slices: 265

name: 318745
IA_minz: 492
slices: 699

name: 318833
IA_minz: 461
slices: 699

name: 322255
IA_minz: 132
slices: 265

name: 325745
IA_minz: 117
slices: 265

name: 329533T
IA_minz: 464
slices: 673

name: 33108
IA_minz: 62
slices: 265

name: 331876
IA_minz: 484
slices: 

In [11]:
# save csv
name_attribute = ['id','slices','IA_voxels','IA_minz','IA_size']
save_csv(csv_info_list,'All_Data_Info.csv',name_attribute)


# save pkl
save_pickle(all_properties,'All_Data_Info.pkl')

In [13]:
shutil.copyfile('All_Data_Info.csv',os.path.join(data_file,'All_Data_Info.csv'))
shutil.copyfile('All_Data_Info.pkl',os.path.join(data_file,'All_Data_Info.pkl'))

shutil.copyfile('All_Data_Info.pkl',os.path.join('/home/ubuntu/codes/radiology/2HeadCut','All_Data_Info.pkl'))

'/home/ubuntu/codes/radiology/2HeadCut/All_Data_Info.pkl'